In [ ]:
!pip install roboflow
!pip install ultralytics
from roboflow import Roboflow
from ultralytics import YOLO
import yaml
from IPython.display import clear_output
import os

# --- 1. Descargar tu NUEVO Dataset Aumentado desde Roboflow ---
print("Descargando tu nuevo dataset desde Roboflow...")
rf = Roboflow(api_key="tZR7XZhFvySrbVM3hfH4")
project = rf.workspace("luchini").project("datasetenproceso-irzcf")

# ----- ¡ACCIÓN REQUERIDA! -----
# (la que tiene los aumentos de Roboflow y la división 92/5/3)
version = project.version(11)
dataset = version.download("yolov8")

# --- 2. Corrección Automática del data.yaml (para evitar errores de ruta) ---
data_yaml_path = f'{dataset.location}/data.yaml'
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"

with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output()
print(f"✅ Dataset (versión {version.version}) descargado y listo.")
print(f"La ubicación del config es: {data_yaml_path}")

# --- 3. Cargar el Modelo ---
# Usamos el modelo 'medium', que dio buenos resultados
model = YOLO('yolov8m-seg.pt')

# --- 4. Iniciar el Entrenamiento ---
print("\nIniciando el entrenamiento...")
results = model.train(
   # --- Parámetros Principales ---
   data=data_yaml_path,
   epochs=200,          # Límite alto, pero 'patience' lo detendrá antes
   patience=40,         # Detener si no hay mejora en 40 épocas
   batch=32,            # Lote grande para tu GPU
   imgsz=640,           # Tamaño del lado más largo
   rect=True,           # ¡IMPORTANTE! Activa el entrenamiento rectangular para tus imágenes 640x480
   optimizer='AdamW',

   # --- Parámetros de Guardado ---
   project='/content/drive/MyDrive/YOLOv8_Training',
   name='Coffee_Seg_RoboflowAug_Run', # Nuevo nombre para esta corrida
   exist_ok=True,

   # --- Aumentos "On-the-Fly" (RECOMENDADO MANTENERLOS) ---
   # Estos se aplican SOBRE tus imágenes ya aumentadas de Roboflow
   degrees=15.0,
   translate=0.1,
   scale=0.5,
   shear=5.0,
   flipud=0.5,
   fliplr=0.5,
   hsv_h=0.015,
   hsv_s=0.7,
   hsv_v=0.4,
   mosaic=1.0,          # Técnica clave de YOLO
   mixup=0.1,
   copy_paste=0.1
)

print("\n✅ Entrenamiento completado.")

✅ Dataset (versión 11) descargado y listo.
La ubicación del config es: /content/DatasetEnProceso-11/data.yaml

Descargando el modelo 'Large' (yolov8l-seg.pt)...

Iniciando el entrenamiento con el modelo 'Large'...
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/DatasetEnProceso-11/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det